# Detecção de fakenews

- Baseado em https://www.youtube.com/watch?v=UMw5-PhRZS0
- dataset https://github.com/recogna-lab/datasets/tree/master/FakeRecogna

In [1]:
#importando bibliotecas
import pandas as pd 
import numpy as np 
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/daniellucas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
print(stopwords.words('portuguese'))

['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'fui', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvésse

In [6]:
#dataset
df = pd.read_excel('dataset/FakeRecogna.xlsx')

In [7]:
#verificando os nulos
df.isnull().sum()

Titulo         31
Subtitulo    6323
Noticia         1
Categoria       1
Data          352
Autor          17
URL             1
Classe          1
dtype: int64

In [8]:
#retirando os nulos
df.dropna(inplace=True)

In [9]:
#verificando os nulos novamente
df.isnull().sum()

Titulo       0
Subtitulo    0
Noticia      0
Categoria    0
Data         0
Autor        0
URL          0
Classe       0
dtype: int64

In [10]:
#juntando os campos
df['all']=df['Titulo']+' '+df['Autor']+' '+df['Noticia']

In [11]:
#verificando a junção dos campos
pd.options.display.max_colwidth = 150

print(df['all'])

0        \n\nPapa Francisco foi preso sob acusação de tráfico de crianças e fraudes #boato \n \nEdgard Matsuki  apagão vaticano papar presar acusação tráfi...
7        \n\nPrograma Renda Brasil +Crédito, com crédito de R$ 50 mil, é lançado em site no WhatsApp #boato \n \nEdgard Matsuki  o cadastramento programar ...
8        Os cheiros que podem desaparecer para sempre  Por BBC  imaginar livrar passar ano guardar estante folhear página amarelar o poeira espalhar ar e c...
9        \n\nVacina contra Covid-19 faz imunidade do corpo diminuir e chance de infecção aumentar #boato \n \nKyene Becker  “ reação vacinar entender facil...
11       \n\nDivaldo Franco divulga mensagem urgente e diz que Brasil corre risco #boato \n \nEdgard Matsuki  mensagem urgente dedivaldofranco “ o brasil c...
                                                                                 ...                                                                          
11892    Papa vai retomar as audiências públic

In [12]:
#criando a função de steming
porter_stemmer = PorterStemmer()

def stemming_tokenizer(df):

    words = re.sub(r"[^A-Za-z0-9]", " ", df).lower().split()
    words = [porter_stemmer.stem(word) for word in words if not words in stopwords.words('portuguese')]
    words = ' '.join(words)
    return words

In [13]:
df['features'] = df['all'].apply(stemming_tokenizer)

In [14]:
print(df['features'])

0        papa francisco foi preso sob acusa o de tr fico de crian as e fraud boato edgard matsuki apag o vaticano papar presar acusa o tr fico crian a e fr...
7        programa renda brasil cr dito com cr dito de r 50 mil lan ado em site no whatsapp boato edgard matsuki o cadastramento programar render brasil cr ...
8        os cheiro que podem desaparec para sempr por bbc imaginar livrar passar ano guardar estant folhear p gina amarelar o poeira espalhar ar e come ar ...
9        vacina contra covid 19 faz imunidad do corpo diminuir e chanc de infec o aumentar boato kyen becker rea o vacinar entend facilidad pessoa infectar...
11       divaldo franco divulga mensagem urgent e diz que brasil corr risco boato edgard matsuki mensagem urgent dedivaldofranco o brasil correr riscar esp...
                                                                                 ...                                                                          
11892    papa vai retomar as audi ncia p blica

In [15]:
#separando em features e "labels"
x = df['features'].values
y = df['Classe'].values

In [16]:
#convertendo de texto para numeros
vectorizer = TfidfVectorizer()
vectorizer.fit(x)

TfidfVectorizer()

In [17]:
x = vectorizer.transform(x)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(x, y , test_size=0.2, stratify=y, random_state=42 )

In [19]:
#treinando o modelo
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression()

In [20]:
#previsão
pred = lr.predict(X_test)

acc = accuracy_score(y_test, pred)

f'acurácia:{ acc * 100:.2f}'

'acurácia:93.96'

In [21]:
import pickle

pickle.dump(lr, open("model", "wb"))

In [22]:
model = pickle.load(open("model", "rb"))

In [23]:
import newspaper

def getNewsArticle(url): 
  article = newspaper.Article(url, language='pt')
  
  article.download()
  article.parse()  

  return article.title +' '+' '.join(article.authors)+' '+article.text

In [24]:
news = getNewsArticle("https://g1.globo.com/economia/noticia/2023/03/20/calendario-bolsa-familia-caixa-comeca-o-pagamento-nesta-segunda-veja-quem-recebe.ghtml")

In [25]:
sampleDf = pd.DataFrame({ "features": [news] })
sampleDf["features"] = sampleDf["features"].apply(stemming_tokenizer)

In [26]:
sample = vectorizer.transform(sampleDf["features"].values)

In [27]:
model.predict(sample)

array([1.])